In [3]:
!pip install pyodbc
import pyodbc
import pandas as pd

import warnings
warnings.filterwarnings("ignore", message="pandas only supports SQLAlchemy connectable")

In [4]:
#connect
try:
    conn = pyodbc.connect('Driver={SQL Server};'
    'SERVER=DESKTOP-PB0KRHD\\SQLEXPRESS;' 
    'DATABASE=States;'
    'Trusted_Connection=yes;')
    print('connected')
except:
    print("failed")

connected


In [5]:
#1.Create a Dataframe showing Quarter wise rainfall with annual rainfall
rain = 'SELECT * FROM RAIN' #fetch from database
rainfall_df = pd.read_sql_query(rain, conn)

rainfall_df['Q1'] = rainfall_df[['JAN', 'FEB', 'MAR']].sum(axis=1)
rainfall_df['Q2'] = rainfall_df[['APR', 'MAY', 'JUN']].sum(axis=1)
rainfall_df['Q3'] = rainfall_df[['JUL', 'AUG', 'SEP']].sum(axis=1)
rainfall_df['Q4'] = rainfall_df[['OCT', 'NOV', 'DEC']].sum(axis=1)

quarterly_rainfall_df = rainfall_df[['YEAR', 'SUBDIVISION', 'Q1', 'Q2', 'Q3', 'Q4', 'ANNUAL']]
print("Quarter wise rainfall with annual rainfall:\n", quarterly_rainfall_df)

#the majority of rainfall occurs in Q3 (July to September), highlighting the monsoon season's impact. 
#For example, in 1901, out of 554.3 mm of annual rainfall, 463.7 mm fell in Q3.
#and q1 with the lowest

Quarter wise rainfall with annual rainfall:
      YEAR         SUBDIVISION         Q1          Q2          Q3          Q4  \
0    1901      GUJARAT REGION   4.800000   68.899999  463.699991   16.900000   
1    1901  MAHARASHTRA REGION  27.099999  174.699995  493.699997   73.599997   
2    1902      GUJARAT REGION   3.900000   34.399999  810.000015   15.700000   
3    1902  MAHARASHTRA REGION   7.900000  117.399999  494.899994  163.800003   
4    1903      GUJARAT REGION   1.800000   42.400001  838.099991    5.400000   
..    ...                 ...        ...         ...         ...         ...   
225  2013  MAHARASHTRA REGION   6.200000  224.099994  669.099991   63.099999   
226  2014      GUJARAT REGION   6.000000   13.900000  681.200012    4.600000   
227  2014  MAHARASHTRA REGION  33.699999   81.299999  638.599998   84.400000   
228  2015      GUJARAT REGION   7.900000  127.099997  485.500015    2.500000   
229  2015  MAHARASHTRA REGION  43.400001  211.000000  325.500000   64.60000

C:\Users\Admin\AppData\Local\Temp\ipykernel_5716\916991861.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  rainfall_df = pd.read_sql(rain, conn)


In [6]:
#2. Separate a Dataframes of Gujarat and Maharashtra Rainfall
gujarat_rainfall_df = rainfall_df[rainfall_df['SUBDIVISION'] == 'GUJARAT REGION']
maharashtra_rainfall_df = rainfall_df[rainfall_df['SUBDIVISION'] == 'MAHARASHTRA REGION']

print("\nGujarat Rainfall DataFrame:\n", gujarat_rainfall_df.head(2))
print("\nMaharashtra Rainfall DataFrame:\n", maharashtra_rainfall_df.head(2))


Gujarat Rainfall DataFrame:
    YEAR     SUBDIVISION  JAN  FEB  MAR  APR  MAY        JUN         JUL  \
0  1901  GUJARAT REGION  4.2  0.0  0.6  1.6  7.0  60.299999  240.199997   
2  1902  GUJARAT REGION  3.9  0.0  0.0  0.6  1.0  32.799999  229.800003   

          AUG         SEP   OCT  NOV   DEC      ANNUAL   Q1         Q2  \
0  205.399994   18.100000  16.6  0.0   0.3  554.299988  4.8  68.899999   
2  299.000000  281.200012   2.3  1.5  11.9  863.900024  3.9  34.399999   

           Q3    Q4  
0  463.699991  16.9  
2  810.000015  15.7  

Maharashtra Rainfall DataFrame:
    YEAR         SUBDIVISION        JAN  FEB  MAR        APR   MAY         JUN  \
1  1901  MAHARASHTRA REGION  18.799999  0.6  7.7  36.599998  30.4  107.699997   
3  1902  MAHARASHTRA REGION   7.800000  0.0  0.1   5.000000   9.8  102.599998   

          JUL         AUG         SEP        OCT   NOV        DEC  ANNUAL  \
1  215.899994  194.100006   83.699997  68.699997   4.4   0.500000   769.0   
3  210.899994  114.5000

In [7]:
#3 use Year as index in Gujarat Dataframe of Rainfall
gujarat_rainfall_df.set_index('YEAR', inplace=True)
print("\nGujarat Rainfall DataFrame with 'Year' as index:\n", gujarat_rainfall_df.head())
# Access data for the year 1901
#data_for_1901 = gujarat_rainfall_df.loc[1902]
#print(data_for_1901)


Gujarat Rainfall DataFrame with 'Year' as index:
          SUBDIVISION  JAN   FEB        MAR  APR   MAY        JUN         JUL  \
YEAR                                                                           
1901  GUJARAT REGION  4.2   0.0   0.600000  1.6   7.0  60.299999  240.199997   
1902  GUJARAT REGION  3.9   0.0   0.000000  0.6   1.0  32.799999  229.800003   
1903  GUJARAT REGION  0.3   0.1   1.400000  0.0  12.3  30.100000  452.899994   
1904  GUJARAT REGION  0.8  10.6  16.799999  0.2   3.9  48.299999  194.800003   
1905  GUJARAT REGION  0.1   0.7   1.100000  0.3   0.0  20.100000  668.299988   

             AUG         SEP   OCT  NOV   DEC      ANNUAL    Q1         Q2  \
YEAR                                                                         
1901  205.399994   18.100000  16.6  0.0   0.3  554.299988   4.8  68.899999   
1902  299.000000  281.200012   2.3  1.5  11.9  863.900024   3.9  34.399999   
1903  202.000000  183.199997   5.4  0.0   0.0  887.599976   1.8  42.400001  

In [8]:
#4. Print the statistics of Gujarat Population Dataframe
gujarat_query = 'SELECT population FROM GUJARAT'
gujarat_pop_df = pd.read_sql_query(gujarat_query, conn)

pd.options.display.float_format = '{:.2f}'.format
# Display statistics of the population column 
print("\nStatistics of Gujarat Population Column:\n", gujarat_pop_df['population'].describe(include='all'))



Statistics of Gujarat Population Column:
 count         12.00
mean    25363689.58
std     17513887.28
min      9094748.00
25%     11161118.25
50%     18448003.50
75%     35891744.75
max     60439692.00
Name: population, dtype: float64


C:\Users\Admin\AppData\Local\Temp\ipykernel_5716\874720019.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  gujarat_pop_df = pd.read_sql(gujarat_query, conn)


In [9]:
#5. Print the growth rate of maharashtra population in year 2001
maharashtra_query = 'SELECT * FROM MAHARASHTRA'
maharashtra_pop_df = pd.read_sql_query(maharashtra_query, conn) 
growth_rate_2001 = maharashtra_pop_df.query('Year == 2001')['GrowthRate'].iloc[0]
conn.close
print(f"\nGrowth rate of Maharashtra population in 2001: {growth_rate_2001}%")


Growth rate of Maharashtra population in 2001: 22.729999542236328%


C:\Users\Admin\AppData\Local\Temp\ipykernel_5716\920297177.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  maharashtra_pop_df = pd.read_sql(maharashtra_query, conn)
